In [0]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms
import matplotlib.pyplot as plt
import time
from glob import glob
import numpy as np
from PIL import Image
import torchvision

In [0]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
from PIL import Image
import os
from pathlib import Path

In [0]:
from torch.utils.data import Dataset, DataLoader

In [0]:
class VAEDataset(Dataset):
    def __init__(self, fpath):
        self.fnames = list(Path(fpath).iterdir())
        self.num_files = len(self.fnames)
        
    def __len__(self):
        return self.num_files
    
    def __getitem__(self, index):
        fname = str(self.fnames[index])
        x = imread(fname)
        x = resize(x)
        x = totensor(x)
        return x, x

In [0]:
def imread(fname):
    return Image.open(fname)

In [0]:
from torch.utils.data import Dataset, DataLoader

No need to run the following two lines, This is just for google colab.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!ls "/content/drive/My Drive"

 Ag_Census_Map_data_07172015.gsheet  'SD Travel Plan.gdoc'
'Colab Notebooks'		      test
 data				     '无标题文档 (1).gdoc'
 data.zip			     '无标题文档 (2).gdoc'
 draft.gdoc			      无标题文档.gdoc
 ikea.gdoc			      量量.gsheet
 imgs


1. Load the data

In [0]:
train_ds = VAEDataset('/content/drive/My Drive/test')

In [10]:
len(train_ds)

2202

In [0]:
learning_rate = 1e-3
batch_size = len(train_ds)

In [0]:
sz = 512

In [0]:
totensor = transforms.ToTensor()
resize = transforms.Resize(sz)

In [0]:
train_loader = DataLoader(train_ds, batch_size=batch_size)

In [14]:
train_loader

Maybe change the hyperparameters for better performance.

In [0]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder,self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=5),
            nn.ReLU(True),
            nn.Conv2d(8,16,kernel_size=5),
            nn.ReLU(True))
        self.decoder = nn.Sequential(             
            nn.ConvTranspose2d(16,8,kernel_size=5),
            nn.ReLU(True),
            nn.ConvTranspose2d(8,3,kernel_size=5),
            nn.ReLU(True),
            nn.Sigmoid())
    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [0]:
num_epochs = 10 #you can go for more epochs, I am using a mac

In [0]:
model = Autoencoder()
distance = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),weight_decay=1e-5)

In [21]:
for epoch in range(num_epochs):
  for data in next(iter(train_loader)) :
      img = data
      img = Variable(img) #.cpu()
      # ===================forward=====================
      output = model(img)
      loss = distance(output, img)
      # ===================backward====================
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
  # ===================log========================
  print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss))

epoch [1/10], loss:0.1446
epoch [2/10], loss:0.1111
epoch [3/10], loss:0.0853
epoch [4/10], loss:0.0869
epoch [5/10], loss:0.0920
epoch [6/10], loss:0.0924
epoch [7/10], loss:0.0861
epoch [8/10], loss:0.0638
epoch [9/10], loss:0.0420
epoch [10/10], loss:0.0309


2. Get embeddings

In [0]:
encoder = model.encoder

In [0]:
x,x = next(iter(train_loader)) 

In [0]:
emb = model.encoder(x)

In [0]:
emb2 = emb.data.numpy()

In [26]:
emb2.shape

(128, 16, 504, 504)

In [0]:
emb_final = emb2.reshape(-1, 16*504*504) #resize

Then, combine the embeddings with `train_ds.fnames` and merge with the products table and ins_post table.

In [0]:
# Then we can fit the embeddings into KNN model.